In [ ]:
##################################################################################################################################################
#                                          Data Fetching 
#################################################################################################################################################

import os
import datetime
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark import SQLContext
from pyspark import HiveContext
from pyspark.sql.functions import col
from pyspark.sql.functions import udf
from pyspark.sql.types import *

from pyspark.sql.functions import *

conf = SparkConf().setAppName("Future Data Prep")

# dev
conf.setMaster("spark://00.0.00.00:0000")
conf.set("spark.cassandrconnection.host", "00.0.00.00")

#sc = SparkContext(conf=conf)
hc = HiveContext(sc)
ssc = SQLContext(sc)

# asset_info data

asset_info_tbl = hc.read.format("org.apache.spark.sql.cassandra").option("table", "cs_fuel_asset_cluster").option("keyspace", "advanced_analytics").load().filter(" corp_cd='FA'")

asset_info_tbl = hc.read.format("filodb.spark").option("dataset", "cs_fuel_asset_cluster").option("database", "advanced_analytics_fdb").load().filter(" corp_cd='FA'")
#cli_no in ('7325','3056','3921') and

asset_info_tbl.registerTempTable("asset_info")

eia_tbl = hc.read.format("filodb.spark").option("dataset", "fuel_eia_series").option("database", "advanced_analytics_fdb").load()
eia_tbl.registerTempTable("eia")
eia_info = hc.sql("select region, fuel_type, fuel_cost_amt/100 as fuel_cost_amt,period_name from eia")

'''asset_df = hc.sql("select corp_cd, cli_no,unit_no, spin_asset_id, invy_stat_cd,tank_capacity_imp,vndr_veh_class_cd_index, \
body_style_imp_index,no_of_cyl_imp,engine_cd_imp,title_inhouse_ind_index,model_cd_index, \
acq_purch_typ_cd_imp_index,cap_cost,drive_imp_index,st_prov_nm, \
drvd_model_index,drvd_make_index,engine_cd_imp_index,gvw_imp, \
mos_in_svc,no_doors_imp_index,series_index, \
prod_class_cd,tire_imp_index,age_of_vehicle \
,mfg_cd_imp_index,clust_no from asset_info")'''
 
asset_df= asset_info_tbl
##############################################################################################################################################
#                                                Missing Imputation
##############################################################################################################################################
asset_df = asset_df.na.fill('212121')

#############################################################################################################################################
###   									Miles Prediction ,MPG and Product_Quantity
#############################################################################################################################################
from pyspark import SparkContext, SparkConf
from pyspark import SQLContext
from pyspark import HiveContext
from pyspark.ml.regression import LinearRegression
from pyspark.sql.types import *
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col
from pyspark.sql.functions import col, udf
from pyspark.sql.functions import col, udf
import numpy as np
import pandas as pd
from pyspark.sql.types import *
hc = HiveContext(sc)
ssc = SQLContext(sc)

mile_tbl = hc.read.format("filodb.spark").option("dataset", "estimates").option("database", "advanced_analytics_fdb").load().filter("corp_cd='FA'   and  period_name in (201701,201702,201703,201704,201705,201706,201707,201708,201709,201710,201710,201711,201712) and  estimate_type=2")
#and cli_no in ('7325','3056','17595','308261')

mile_tbl.registerTempTable("mile")

asset_info_tbl = hc.read.format("filodb.spark").option("dataset", "asset_info").option("database", "core_fdb").load().filter("curr_ast_cli_flag='Y'").filter("corp_cd='FA' ")

#and cli_no in ('7325','3056','17595','308261')
asset_info_tbl.registerTempTable("asset_info")

miles_info = hc.sql("select mile.corp_cd, mile.cli_no,mile.inv_smry_dt,mile.spin_asset_id, mile.estimate_type,mile.period_name, mile.lou_est_value,mile.monthly_est_value,asset.city_loc_cd from mile mile inner join asset_info asset on asset.corp_cd=mile.corp_cd and asset.cli_no=mile.cli_no and asset.spin_asset_id=mile.spin_asset_id")
#miles_info.registerTempTable("mile_info")
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++#
#										Column Creation
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++#

miles_info = miles_info.withColumn("period_name_f",miles_info["period_name"].cast(DoubleType()).alias("period_name"))
#asset_fuel_2016 = asset_fuel_2016.drop('period_name')

miles_info= miles_info.withColumn('month_s', miles_info.period_name_f.substr(5,6))
miles_info= miles_info.withColumn('month_num', miles_info.month_s.cast(DoubleType()))

miles_info= miles_info.withColumn('year_s', miles_info.period_name_f.substr(1,4))
miles_info= miles_info.withColumn('year_num', miles_info.month_s.cast(DoubleType()))

miles_info= miles_info.withColumn('client_no', miles_info.cli_no.cast(DoubleType()))

miles_info= miles_info.withColumn('city_cd', miles_info.city_loc_cd.cast(DoubleType()))

miles_info.registerTempTable("mile_info")

#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++#
#								creating the vector for all predictors
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++#

#cli_no: string, inv_smry_dt: timestamp, spin_asset_id: int, estimate_type: int, period_name: int, lou_est_value: double, #monthly_est_value: double, city_loc_cd: string]

miles_tl = miles_info.select('client_no','inv_smry_dt','spin_asset_id','period_name','month_num', \
'monthly_est_value','city_cd')

from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, PolynomialExpansion, VectorIndexer

feat_list = ['client_no','spin_asset_id','month_num','city_cd']

assembler = VectorAssembler( inputCols = feat_list, outputCol = "features")

#miles_data = assembler.transform( miles_tl)

#miles_data = miles_data.withColumn( "label", miles_data.monthly_est_value.cast( 'double' ) )

#miles_data.select( "features", "label" ).show( 5 )

#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++#
#								Feature Engg. 
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++#

from pyspark.sql.functions import lag, col, lead
from pyspark.sql.window import Window

########################## Average of Miles in last 3 months

w = Window().partitionBy("cli_no","spin_asset_id").orderBy("inv_smry_dt")

miles_info = miles_info.withColumn("avg_miles_last_3months",
                                       ( lag("monthly_est_value", 1).over(w)
									   + lag("monthly_est_value", 2).over(w)
										+ lag("monthly_est_value", 3).over(w)) /3  )
									   
									   
									   
#w = Window().partitionBy("Store").orderBy("Date")									   
###################Average miles per last 6 months
w = Window().partitionBy("cli_no","spin_asset_id").orderBy("inv_smry_dt")
miles_info = miles_info.withColumn("avg_miles_last_6months",
                                       ( lag("monthly_est_value", 1).over(w)
                                        + lag("monthly_est_value", 2).over(w)
                                        + lag("monthly_est_value", 3).over(w)
                                        + lag("monthly_est_value", 4).over(w)
                                        + lag("monthly_est_value", 5).over(w)
										+ lag("monthly_est_value", 6).over(w))/6)
										
###################Average miles per last 2 months

w = Window().partitionBy("cli_no","spin_asset_id").orderBy("inv_smry_dt")								
miles_info = miles_info.withColumn("avg_miles_last_2months",
                                       ( lag("monthly_est_value", 1).over(w)
                                        + lag("monthly_est_value", 2).over(w))/2)
										
										
###################Average miles 									
avg_miles_cli_spinid_by_month = miles_info.groupBy( "cli_no","spin_asset_id" ).avg( "monthly_est_value" )

#avg_miles_cli_spinid_by_month.show( 5 )

miles_new_df = miles_info.join( avg_miles_cli_spinid_by_month, (miles_info.cli_no==avg_miles_cli_spinid_by_month.cli_no) & (miles_info.spin_asset_id==avg_miles_cli_spinid_by_month.spin_asset_id), "inner" ).drop(avg_miles_cli_spinid_by_month.spin_asset_id).drop(avg_miles_cli_spinid_by_month.cli_no)

miles_new_df = miles_new_df.withColumnRenamed('AVG(monthly_est_value)', "avg_monthly_value")									   
									   
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++#
#								   Variable Selection
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++#
								   
									   
miles_new_df = miles_new_df.select('client_no','spin_asset_id','city_cd','avg_miles_last_3months','avg_miles_last_6months','avg_miles_last_2months',"avg_monthly_value","month_num","monthly_est_value")
 
miles_new_df=miles_new_df.fillna(0) 
      
feat_list = feat_list + ['avg_miles_last_3months','avg_miles_last_6months','avg_miles_last_2months',"avg_monthly_value"]

assembler = VectorAssembler( inputCols = feat_list, outputCol = "features")
miles_new_df = assembler.transform( miles_new_df )
miles_new_df = miles_new_df.withColumn( "label", miles_new_df.monthly_est_value.cast( 'double' ) )
#miles_new_df.select( "features", "label" ).show( 5 )

seed = 42

train_df, test_df = miles_new_df.randomSplit( [0.7, 0.3], seed = seed )

linreg = LinearRegression(maxIter=500, regParam=0.0)
lm = linreg.fit( train_df )

lm.intercept

lm.coefficients

y_pred = lm.transform( test_df )
#y_pred.select( 'features', 'label', 'prediction' ).show( 5 )

r2_evaluator = RegressionEvaluator(labelCol="label",
                            predictionCol="prediction",
                            metricName="r2" )

lm_r2 = r2_evaluator.evaluate( y_pred )
lm_r2

'''rmse_evaluator = RegressionEvaluator(labelCol="label",
                            predictionCol="prediction",
                            metricName="rmse" )

lm_rmse = rmse_evaluator.evaluate(y_pred)
lm_rmse'''
                                                   
##+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#   								Future Data Prep
#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++       
import numpy as np 
import pandas as pd 
import itertools

'''def cartesian(df1, df2):
    rows = itertools.product(df1.iterrows(), df2.iterrows())
    df = pd.DataFrame(left.append(right) for (_, left), (_, right) in rows)
    return df.reset_index(drop=True)'''

future_id = hc.sql("select distinct  client_no,spin_asset_id,city_cd from mile_info")

futur = future_id.toPandas()

import pandas as pd
month_date = '2018-01-01'
date_data = pd.DataFrame(pd.date_range(month_date, periods=12, freq='M'),columns=["date"])
date_data['year'] = pd.DatetimeIndex(date_data['date']).year
date_data['month_num'] = pd.DatetimeIndex(date_data['date']).month
date_data["period_name"] = date_data["year"].map(str) + date_data.month_num.map("{:02}".format)

futur['key'] = 0
date_data['key'] = 0


future_date = futur.merge(date_data, how='outer')

future_data =  sqlContext.createDataFrame(future_date)

future_data = future_data.join(miles_new_df['client_no','month_num','spin_asset_id','avg_monthly_value','avg_miles_last_2months','avg_miles_last_3months','avg_miles_last_6months'],(future_data.client_no==miles_new_df.client_no) & (future_data.spin_asset_id==miles_new_df.spin_asset_id) & (future_data.month_num==miles_new_df.month_num) ).drop(miles_new_df.spin_asset_id).drop(miles_new_df.client_no).drop(miles_new_df.month_num)

feat_list1 = ['client_no','spin_asset_id','month_num','city_cd','avg_miles_last_3months','avg_miles_last_6months','avg_miles_last_2months',"avg_monthly_value"]

assembler = VectorAssembler( inputCols = feat_list1, outputCol = "features")

miles_fut_data = assembler.transform( future_data)

miles_predicted = lm.transform(miles_fut_data)
#miles_predicted

miles_predicted = miles_predicted.withColumnRenamed('prediction', "monthly_est_value")
miles_predicted = miles_predicted.withColumnRenamed('client_no', "cli_no")

#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++#
#												MPG Misssing Value Imputation
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++#

epa_tbl = hc.read.format("org.apache.spark.sql.cassandra").option("table", 
                                                                  "fuel_epa_mpg").option("keyspace", "advanced_analytics").load()
epa_tbl.registerTempTable("epa")


#epa_info = hc.sql("select epa.corp_cd, epa.cli_no, epa.spin_asset_id, epa.citympg, epa.hwympg, epa.combmpg from epa where  citympg is not null")

asset_info_tbl = hc.read.format("filodb.spark").option("dataset", 
                                                       "asset_info").option("database", "core_fdb").load().filter("curr_ast_cli_flag='Y'").filter("corp_cd='FA' ")

#and cli_no in ('7325','3056','17595','308261')
asset_info_tbl.registerTempTable("asset_info")

mpg_imputed_all = hc.sql("select asset.corp_cd \
,asset.cli_no \
,asset.spin_asset_id \
,epa.citympg \
,epa.combmpg \
,epa.hwympg \
,asset.prod_class_cd \
,case \
when epa.combmpg not in (0,NULL,'Infinity')  then epa.combmpg \
when epa.combmpg in (0,NULL,'Infinity') and epa_avg.combavg_m not in (0,NULL,'Infinity')   then epa_avg.combavg_m \
when epa.combmpg in (0,NULL,'Infinity') and epa_avg.combavg_m in (0,NULL,'Infinity') and epa_prd_reg_avg.combavg_m not in (0,NULL,'Infinity')  then epa_prd_reg_avg.combavg_m  \
when epa.combmpg in (0,NULL,'Infinity') and epa_avg.combavg_m in (0,NULL,'Infinity') and epa_prd_reg_avg.combavg_m  in (0,NULL,'Infinity') and epa_overll_avg.combavg_m not in (0,NULL,'Infinity') then epa_overll_avg.combavg_m \
end as mpg \
,epa_avg.combavg_m  \
,epa_prd_reg_avg.combavg_m  \
,epa_overll_avg.combavg_m  \
from epa epa  \
right join asset_info asset \
on asset.corp_cd=epa.corp_cd and asset.cli_no=epa.cli_no and asset.spin_asset_id=epa.spin_asset_id \
left outer join  \
(select prod_class_cd,drvd_model,engine_cd,geo_region_cd,avg(combmpg) as combavg_m \
from epa epa \
inner join asset_info asset \
on asset.corp_cd=epa.corp_cd and asset.cli_no=epa.cli_no and asset.spin_asset_id=epa.spin_asset_id \
where combmpg not in (0,NULL,'Infinity') \
group by prod_class_cd,drvd_model,engine_cd,geo_region_cd) epa_avg \
on \
epa_avg.prod_class_cd = asset.prod_class_cd \
and epa_avg.drvd_model = asset.drvd_model \
and epa_avg.engine_cd = asset.engine_cd \
and epa_avg.geo_region_cd = asset.geo_region_cd \
left outer join \
(select prod_class_cd,geo_region_cd,avg(combmpg) as combavg_m \
from fdb_adv_ana_fuel_epa_mpg epa \
inner join fdb_core_asset_info asset \
on asset.corp_cd=epa.corp_cd and asset.cli_no=epa.cli_no and asset.spin_asset_id=epa.spin_asset_id \
where combmpg not in (0,NULL,'Infinity') \
group by prod_class_cd,geo_region_cd) epa_prd_reg_avg \
on \
epa_prd_reg_avg.prod_class_cd = asset.prod_class_cd \
and epa_prd_reg_avg.geo_region_cd = asset.geo_region_cd \
left outer join \
(select prod_class_cd,avg(combmpg) as combavg_m \
from fdb_adv_ana_fuel_epa_mpg epa \
inner join fdb_core_asset_info asset \
on asset.corp_cd=epa.corp_cd and asset.cli_no=epa.cli_no and asset.spin_asset_id=epa.spin_asset_id \
where combmpg not in (0,NULL,'Infinity') \
group by prod_class_cd) epa_overll_avg \
on \
epa_overll_avg.prod_class_cd= asset.prod_class_cd \
where asset.corp_cd = 'FA' ")

#and asset.cli_no in ('7325','3056','17595','308261')

#from functools import reduce  # For Python 3.x
#from pyspark.sql import DataFrame

#def unionAll(*dfs):
    #return reduce(DataFrame.unionAll, dfs)

#mpg_imputed_all =unionAll(mpg_imputed,mpg_org)
 
#imputed_mpg.registerTempTable("imputed_mpg")
#chk= hc.sql("select corp_cd,cli_no,spin_asset_id,prod_class_cd,drvd_model,engine_cd,geo_region_cd,combmpg,combavg_m \
#from imputed_mpg where combavg_m in (0,NULL,'Infinity')")

#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++#
#                                      Product Quantity Prep
#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++#

miles_mpg = miles_predicted['cli_no',
                            'spin_asset_id','period_name',
                            'monthly_est_value'].join(mpg_imputed_all['spin_asset_id',
                                                                      'prod_class_cd', 'mpg'],(miles_predicted.spin_asset_id == mpg_imputed_all.spin_asset_id),"left_outer").drop(mpg_imputed_all.spin_asset_id)

#sqlContext.registerDataFrameAsTable(miles_mpg, "miles_mpg_d")
#miles_mpg.registerTempTable("miles_mpg_d")
#miles_mpg = hc.sql("select a.corp_cd,a.cli_no,a.spin_asset_id,a.prod_class_cd,a.drvd_model,a.engine_cd,a.geo_region_cd,b.combavg_m as #combmpg \
#from miles_mpg_d a left join epa_overll_avg b  where  a.prod_class_cd=b.prod_class_cd  and  a.combmpg in (0,NULL,'Infinity')")

miles_mpg = miles_mpg.withColumn('prod_qnty', miles_mpg.monthly_est_value/miles_mpg.mpg)

############################################################Region Creation
# add region
PADD = {
    'East Coast': ['ME', 'NH', 'MA', 'RI', 'CT', 'NY', 'NJ', 'DE', 'MD', 'VA', 'NC', 'SC', 'GA', 'FL', 'WV', 'PA', 'VT',
                   'DC'],
    'Gulf Coast': ['NM', 'TX', 'AR', 'LA', 'MS', 'AL'],
    'Rocky Mountain': ['ID', 'MT', 'WY', 'CO', 'UT'],
    'Midwest': ['IL', 'IN', 'OH', 'MI', 'WI', 'MO', 'NE', 'MN', 'KS', 'ND', 'SD', 'IA', 'OK', 'TN', 'KY'],
    'West Coast': ['CA', 'OR', 'WA', 'AK', 'NV', 'AZ', 'HI']}

def get_region(s):
    for region, states in PADD.iteritems():
        for state in states:
            if state in s:
                return region

udfgetregion = udf(get_region, StringType())

# create column for abbr version of state
STATE = {'ME': 'MAINE',
         'NH': 'NEW HAMPSHIRE',
         'MA': 'MASSACHUSETTS',
         'RI': 'RHODE ISLAND',
         'CT': 'CONNECTICUT',
         'NY': 'NEW YORK',
         'NJ': 'NEW JERSEY',
         'DE': 'DELAWARE',
         'MD': 'MARYLAND',
         'VA': 'VIRGINIA',
         'NC': 'NORTH CAROLINA',
         'SC': 'SOUTH CAROLINA',
         'GA': 'GEORGIA',
         'FL': 'FLORIDA',
         'WV': 'WEST VIRGINIA',
         'PA': 'PENNSYLVANIA',
         'VT': 'VERMONT',
         'DC': 'distRICT OF COLUMBIA',
         'NM': 'NEW MEXICO',
         'TX': 'TEXAS',
         'AR': 'ARKANSAS',
         'LA': 'LOUISIANA',
         'MS': 'MISSISSIPPI',
         'AL': 'ALABAMA',
         'ID': 'IDAHO',
         'MT': 'MONTANA',
         'WY': 'WYOMING',
         'CO': 'COLORADO',
         'UT': 'UTAH',
         'IL': 'ILLINOIS',
         'IN': 'INDIANA',
         'OH': 'OHIO',
         'MI': 'MICHIGAN',
         'WI': 'WISCONSIN',
         'MO': 'MISSOURI',
         'NE': 'NEBRASKA',
         'MN': 'MINNESOTA',
         'KS': 'KANSAS',
         'ND': 'NORTH DAKOTA',
         'SD': 'SOUTH DAKOTA',
         'IA': 'IOWA',
         'OK': 'OKLAHOMA',
         'TN': 'TENNESSEE',
         'KY': 'KENTUCKY',
         'CA': 'CALIFORNIA',
         'OR': 'OREGON',
         'WA': 'WASHINGTON',
         'AK': 'ALASKA',
         'NV': 'NEVADA',
         'AZ': 'ARIZONA',
         'HI': 'HAWAII'}

def get_state_abbr(s):
    for state_abbr, statefull in STATE.items():
        if s == statefull:
            return state_abbr

udfgetabr = udf(get_state_abbr, StringType())

asset_df = asset_df.withColumn("state", udfgetabr(asset_df["st_prov_nm"]))

asset_df = asset_df.withColumn("region",udfgetregion(asset_df['state']))
############################################################################################################################################
#    Joining Miles,MPG  and EIA Price  Data
############################################################################################################################################

asset_mpg_df = asset_df.join(miles_mpg['spin_asset_id', 'period_name','monthly_est_value','prod_qnty','mpg'], 
                             (asset_df.spin_asset_id == asset_df.spin_asset_id),
                             "inner").drop(miles_mpg.spin_asset_id)

asset_mpg_df = asset_mpg_df.join(eia_info['region','period_name','fuel_cost_amt'], 
                                 (eia_info.region == asset_mpg_df.region) 
                                 & (eia_info.period_name == asset_mpg_df.period_name),
                                 "left_outer").drop(asset_mpg_df.region).drop(eia_info.period_name)

####################################################################################################################################################   creating a Column with month name
#########################################################################################################################################
asset_mpg_df = asset_mpg_df.withColumn("period_name_f",asset_mpg_df["period_name"].cast(DoubleType()).alias("period_name"))
#asset_fuel_2016 = asset_fuel_2016.drop('period_name')

asset_mpg_df= asset_mpg_df.withColumn('month_s', asset_mpg_df.period_name_f.substr(5,6))
asset_mpg_df= asset_mpg_df.withColumn('month_num', asset_mpg_df.month_s.cast(DoubleType()))

#asset_mpg_df= asset_mpg_df.withColumn('no_of_doors_imp', asset_mpg_df.no_doors_imp.cast(DoubleType()))

asset_mpg_df = asset_mpg_df.withColumn("month", when((col('month_s') == "01.0"), "Jan") \
                                       .when((col('month_s') == "02.0") , "Feb") \
                                       .when((col('month_s') == "03.0") , "Mar") \
                                       .when((col('month_s') == "04.0"), "Apr") \
                                       .when((col('month_s') == "05.0"), "May") \
                                       .when((col('month_s') == "06.0"), "Jun") \
                                       .when((col('month_s') == "07.0"), "Jul") \
                                       .when((col('month_s') == "08.0"), "Aug") \
                                       .when((col('month_s') == "09.0"), "Sep") \
                                       .when((col('month_s') == "10.0"), "Oct") \
                                       .when((col('month_s') == "11.0"), "Nov") \
                                       .when((col('month_s') == "12.0"), "Dec") \
                                       .otherwise(col("month_s")))

############################################################################################################################################
#  Data  Uploading 

#feature_dpython = feature_data.toPandas()
#feature_dpython.to_csv("/home/dev/python_models/feature_dpython.csv")
 
#feature_data.registerTempTable("feature_data")
run_date = datetime.datetime.now()
asset_mpg_df = asset_mpg_df.withColumn('run_date', lit(run_date))

asset_mpg_df = asset_mpg_df.sort("corp_cd","cli_no")

asset_mpg_df.write.format("org.apache.spark.sql.cassandra").mode('append').options(keyspace="advanced_analytics",table="cs_fuel_asset_future_data").save()
ssc.clearCache()
